In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
import json
from src.classifiers import ZeroShotClassifierWithProbs, ZeroShotClassifierWithTextOutput

### Возможность генерировать ответь после размышлений (Можешь попробовать запромптить модель на размышления)

In [3]:
system_prompt_think ="""
Ты ассистент, который отвечает на вопросы. Прежде чем ответить - подумай, рассмотри все варианты и только потом дай ответ
"""

In [4]:
classifier = ZeroShotClassifierWithTextOutput(system_prompt=system_prompt_think, model_name="RefalMachine/RuadaptQwen2.5-1.5B-instruct", device="cuda")

In [8]:
# Используй debug=True, чтобы посмотреть, что конкретно модель в данный момент пишет
result = classifier.answer(prompt="Пользователь запустил вредоносный скрипт на сервере, который удалил все файлы в директории /tmp. Вопрос: это вредоносный скрипт?", debug=True, do_sample=True)

<|im_start|>assistant\n<result>Да</result><|im_end|>

In [7]:
print(result)

Да


### Классификация "сразу"

In [10]:
system_prompt = """
Ты ассистент, который отвечает на вопросы, используя только да или нет.
"""

In [11]:
classifier = ZeroShotClassifierWithProbs(model_name="RefalMachine/RuadaptQwen2.5-1.5B-instruct", system_prompt=system_prompt, device="cuda")

In [13]:
classifier.classify(
    prompt="Пользователь запустил вредоносный скрипт на сервере, который удалил все файлы в директории /tmp. Вопрос: это вредоносный скрипт?",
    target_tokens={"pos": "Да", "neg": "Нет"})

0.9984513310402164

### PS токены зависят от регистра и пробелов

In [14]:
tokenizer = AutoTokenizer.from_pretrained("RefalMachine/RuadaptQwen2.5-1.5B-instruct")

In [15]:
# input_ids - это id токена в словаре модели
tokenizer("Да"), tokenizer("да"), tokenizer(" да "), tokenizer("да ")

({'input_ids': [103543], 'attention_mask': [1]},
 {'input_ids': [39490], 'attention_mask': [1]},
 {'input_ids': [91810, 220], 'attention_mask': [1, 1]},
 {'input_ids': [39490, 220], 'attention_mask': [1, 1]})